In [ ]:
# 0 1 2 4
# 1 0 3 5
# 2 3 0 6
# 4 5 6 0

# i = 1 : (1,2,4)
# i = 2 : (1,3,5)
# i = 3 : (2,3,6)
# i = 4 : (4,5,6)

#ind = [[0 1 2 4]
#       [1 0 3 5]
#       [2 3 0 6]
#       [4 5 6 0]]

#       6
#  1:s ---- 2 
#  6 |      | 6
#    |      |
#    4 ---- 3:t
#       6
    
#h = zeros(6)
#h[1] = 6
#h[2] = 0
#h[3] = 6
#h[4] = 6
#h[5] = 0
#h[6] = 6
    
#s = 1
#t = 3
#min_cut(s,t,h,ind)

In [273]:
# load data
using DelimitedFiles, LinearAlgebra
dist = readdlm("data42.txt",Int)

V = size(dist)[1]
E = (V-1)*(V) ÷ 2

function index(i,j)
    return (i-2)*(i-1) ÷ 2 + j # Lazy Caterer's sequence + (j-1)
end

ind = zeros(Int, (V,V)) # construct matrix with 2 Floyd's triangle
c = zeros(Int, E) # construct cost vector

for i = 2:V
    for j = 1:i-1
        ind_ij = index(i,j)
        ind[i,j] = ind_ij
        c[ind_ij] = dist[i,j] 
    end
end
ind = ind + transpose(ind)

42×42 Array{Int64,2}:
   0    1    2    4    7   11   16   22  …  596  631  667  704  742  781  821
   1    0    3    5    8   12   17   23     597  632  668  705  743  782  822
   2    3    0    6    9   13   18   24     598  633  669  706  744  783  823
   4    5    6    0   10   14   19   25     599  634  670  707  745  784  824
   7    8    9   10    0   15   20   26     600  635  671  708  746  785  825
  11   12   13   14   15    0   21   27  …  601  636  672  709  747  786  826
  16   17   18   19   20   21    0   28     602  637  673  710  748  787  827
  22   23   24   25   26   27   28    0     603  638  674  711  749  788  828
  29   30   31   32   33   34   35   36     604  639  675  712  750  789  829
  37   38   39   40   41   42   43   44     605  640  676  713  751  790  830
  46   47   48   49   50   51   52   53  …  606  641  677  714  752  791  831
  56   57   58   59   60   61   62   63     607  642  678  715  753  792  832
  67   68   69   70   71   72   73   74   

In [274]:
function min_cut(s,t,h,ind)
    model = Model(with_optimizer(Gurobi.Optimizer))
    set_optimizer_attribute(model, "Presolve", 0)
    set_optimizer_attribute(model, "Heuristics", 0)
    set_optimizer_attribute(model, "Cuts", 0)
    set_silent(model)
    
    E = size(h)[1]
    V = size(ind)[1]
    
    # variables
    @variable(model,0 <= w[1:E])
    @variable(model,u[1:V])
    
    # objective function
    @objective(model, Min, sum(h .* w))
    
    # constraints
    for e = 1:E
        ij = findall(x->(x==e),ind)[1]
        i = ij[1]
        j = ij[2]
        @constraint(model,w[e]>=u[i]-u[j])
        @constraint(model,w[e]>=u[j]-u[i])
    end
    @constraint(model,u[t]-u[s]>=1)
    
    optimize!(model)
    return sum(h.*JuMP.value.(w)), findall(x->x==0.0,JuMP.value.(u))
end

min_cut (generic function with 2 methods)

In [286]:
function delta(S,ind)
    
    V = size(ind)[1]
    E = V*(V-1) ÷ 2 
    
    indices_total = Set{Int}()
    for e = 1:E
        ij = findall(x->(x==e),ind)[1]
        i = ij[1]
        j = ij[2]
                
        if ((i in S) && !(j in S)) || (!(i in S) && (j in S))
            push!(indices_total,e)
        end
    end
    
    return collect(indices_total)
end

delta (generic function with 1 method)

In [288]:
using Gurobi, JuMP

model = Model(with_optimizer(Gurobi.Optimizer))
set_optimizer_attribute(model, "Presolve", 0)
set_optimizer_attribute(model, "Heuristics", 0)
set_optimizer_attribute(model, "Cuts", 0)
set_silent(model)

# define variables and constraints for the initial LP problem
@variable(model,0 <= x[1:E] <= 1)
@objective(model,Min,sum(c.*x))
for i = 1:V 
    indices = filter!(x -> x ≠ 0,ind[:,i])
    @constraint(model,sum(x[indices]) == 2)
end

# initialization
optimize!(model)
opt_x = JuMP.value.(x)

count=0
count2=0
while true
    t=2
    while t ≠ V+1
        cut_val,S = min_cut(1,t,opt_x,ind)
        count2 = count2+1
        if cut_val < 2
            count = count+1
            ind_cons = delta(S,ind)
            @constraint(model,sum(x[ind_cons]) >= 2)
            optimize!(model)
            opt_x = JuMP.value.(x)
            break
        end
        t = t+1
    end
    if t==V+1
        break
    end
end

# we have the root node x_opt
opt_x[findall(x->(x ≠ 1.0) && (x ≠ 0.0),opt_x)]


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only

--------------------------

16-element Array{Float64,1}:
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5